In [1]:
from sklearn import svm,metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import glob, os.path, re

matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows 사용자
# matplotlib.rcParams['font.family'] = 'AppleGothic Gothic' # Mac사용자
matplotlib.rcParams['font.size'] = '10' # 글자크기

In [17]:
# # 파일 저장하기
# import requests
# url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
# res = requests.get(url)
# res.raise_for_status()

# with open("data11.csv","w",encoding="utf8") as f:
#     f.write(res.text)

###### 식용, 독버섯 분류 알고리즘 구현

In [2]:
# 파일 불러오기
df = pd.read_csv('data11.csv',header=None)# header, 맨위 컬럼 제거
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [19]:
target = []
data = []
# attr_list = []

In [20]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [21]:
# 아스키코드로 변환
# # 문자를 숫자로 변환

# # 2가지 데이터가 함께 나온다
# for row_index, row in df.iterrows(): # enumerate와 같다
#     target.append(row.loc[0]) # 결과데이터 저장 p or e
#     row_data = []
#     for v in row.loc[1:]:
#         row_data.append(ord(v)) # 1줄의 데이터를 숫자로 변경해서 list저장
    
#     data.append(row_data) # 1줄 리스트를 리스트에 추가
    
# data
# df2 = pd.DataFrame(data)
# df2

In [ ]:
# 원핫인코딩으로 변환 @@@@@@@@@주석
target = []  # 결과 데이터를 저장할 리스트
data = []    # 원핫인코딩된 데이터를 저장할 리스트
attr_list = []  # 속성에 대한 정보를 저장할 리스트

# 각 행(row)에 대해 반복
for row_index, row in df.iterrows():
    target.append(row.loc[0])  # 결과 데이터(p 또는 e) 저장

    row_data = []  # 한 행의 데이터를 저장할 리스트

    # 각 열(column)에 대해 반복
    for col, v in enumerate(row.loc[1:]):
        if row_index == 0:  # 첫 번째 행인 경우에만 속성 정보를 초기화
            attr = {'dic': {}, 'cnt': 0}  # 속성 정보를 저장할 딕셔너리와 개수를 초기화
            attr_list.append(attr)  # 속성 정보를 리스트에 추가
        else:
            attr = attr_list[col]  # 해당 열의 속성 정보를 가져옴

        # 원핫인코딩 변환을 위한 리스트를 초기화
        d = [0] * 12

        # 속성의 값이 이미 딕셔너리에 존재하는 경우 해당 인덱스를 사용하고,
        # 그렇지 않은 경우 새로운 인덱스를 할당하고 딕셔너리에 추가
        if v in attr['dic']:
            idx = attr['dic'][v]
        else:
            idx = attr['cnt']
            attr['dic'][v] = idx
            attr['cnt'] += 1

        d[idx] = 1  # 해당 인덱스에 1을 할당하여 원핫인코딩을 수행
        row_data += d  # 변환된 데이터를 행 데이터 리스트에 추가

    data.append(row_data)  # 한 행의 데이터를 데이터 리스트에 추가

# 데이터 리스트를 사용하여 새로운 DataFrame 생성
df2 = pd.DataFrame(data)
df2  # 변환된 데이터프레임 출력


In [3]:
# 원핫인코딩으로 변환
target = []
data = []
attr_list = []
for row_index, row in df.iterrows(): # enumerate와 같다
    target.append(row.loc[0]) # 결과데이터 저장 p or e
    row_data = []
    for col,v in enumerate(row.loc[1:]):
        if row_index == 0:
            attr = {'dic':{},'cnt':0} # cnt: 종류의 수, 다른게 몇가지인가
            attr_list.append(attr)
        else:
            attr = attr_list(col) # 1번째방에 입력
            
        # 원핫인코딩 변환
        d = [0]*12
        if v in attr['dic']:
            idx = attr['dic'][v]
            # print(idx)
        else:
            idx = attr['cnt']
            attr['dic'][v] = idx
            # print(attr['dic'][v])
            attr['cnt'] += 1
            
        d[idx] = 1
        row_data += d # 원핫인코딩의 데이터 저장파일, 데이터의 가중치로 정답률이 다를 수 있다.
        
    data.append(row_data) # 1줄 리스트를 리스트에 추가

data[0]    


TypeError: 'list' object is not callable

In [ ]:
# 문자를 숫자로 변환
data = df.iloc[:,1:]
all_list = []
for d in data:
    d_list.append(ord(d))
d_list

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# 데이터 전처리: train, test세트 구현
train_input,test_input,train_target,test_target = train_test_split(data, target)

# 데이터 학습하기
clf = RandomForestClassifier()
clf.svm.SVC()
clf.fit(train_input,train_target)

# 데이터 예측하기
predict = clf.predict(test_input)
print("결과값: ",predict)

# 정답률 구하기
score = clf.score(test_input,test_target)
print("정답률: ",score)

결과값:  ['p' 'e' 'e' ... 'p' 'e' 'e']
정답률:  1.0
